```python
import dagshub
dagshub.init(repo_owner='SaadMalik0011', repo_name='Kidney-Disease-Classification-With-MLOps', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)
  ```

In [4]:
import os

In [5]:
%pwd

'c:\\Users\\PMLS\\Desktop\\Jupyter notebook\\Campusx Codes\\Deep-Learning-Project\\cnnclassifier\\notebooks'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\PMLS\\Desktop\\Jupyter notebook\\Campusx Codes\\Deep-Learning-Project\\cnnclassifier'

```.env
MLFLOW_TRACKING_URI=https://dagshub.com/SaadMalik0011/Kidney-Disease-Classification-With-MLOps.mlflow/
MLFLOW_TRACKING_USERNAME=SaadMalik0011
MLFLOW_TRACKING_PASSWORD=c74fa845dcdcef2b48d107eac0769b0fc7a864fb
```

True

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model("models/training/model.h5")


## 1. Update config.yaml

```yaml
evaluation:
  mlflow_uri: https://dagshub.com/SaadMalik0011/Kidney-Disease-Classification-With-MLOps.mlflow/
```

## 4. Update the entity

In [47]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    params_learning_rate: float
    

## 5. Update the configuration manager src/config/configuration.py

In [48]:
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.utils.common import read_yaml, create_directories, save_json
from src.logger import CustomLogger

class ConfigurationManager:
    def __init__(
        self,
        logger: CustomLogger,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        
        self.logger = logger        
        self.config = read_yaml(config_filepath, logger=self.logger)
        self.params = read_yaml(params_filepath, logger=self.logger)   

        create_directories([self.config.model_artifacts_root], logger=self.logger)
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config.training.trained_model_path,
            training_data=self.config.training.data_path,
            mlflow_uri=self.config.evaluation.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_learning_rate=self.params.LEARNING_RATE
                
        )
        
        return eval_config

## 6. Update the components \[data preprocessing, model training, and so on\]

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from dotenv import load_dotenv

load_dotenv()

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
        
    def test_generator(self, test_generator_logger: CustomLogger):
    
        try:
            
            
            ########## kwargs start ########## 
            
            datagenerator_kwargs = dict(
                rescale=1./255,
                validation_split=0.30
            )
            
            dataflow_kwargs = dict(
                target_size = self.config.params_image_size[:-1],
                batch_size = self.config.params_batch_size,
                interpolation = "bilinear"
            )
            
            test_specific_kwargs = dict(
                directory = self.config.training_data,
                subset="validation",
                shuffle = False,
                class_mode = "categorical"
            )
            
            
            ########## kwargs end ########## 
            

                    
            ########## test datagenerator start ########## 
            test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
            
            self.test_generator = test_datagenerator.flow_from_directory(
                **test_specific_kwargs,
                **dataflow_kwargs
            )
            
            test_generator_logger.save_logs(msg=f"Generated Validation generator successfully with these \ndatagenerator_kwargs: {datagenerator_kwargs},\ndataflow_kwargs: {test_specific_kwargs, dataflow_kwargs}", log_level='info')
            
            ########## test datagenerator end ##########
            
        except Exception as e:
            test_generator_logger.save_logs(msg=f"Error in creating train and validation Generators. Error: {e}",log_level="error")
            raise e
        
        
    @staticmethod
    def load_model(path: Path, logger: CustomLogger) -> tf.keras.Model:
        try:
            model = tf.keras.models.load_model(path)
            logger.save_logs(msg=f"Model loaded from {path} successfully.", log_level="info")
        except Exception as e:
            logger.save_logs(msg=f"Failed to load model from {path} with Error: {e}", log_level="error")
            raise e
        
        return model
    
    def evaluate(self, evaluation_logger:CustomLogger):
        self.model = self.load_model(self.config.path_of_model, evaluation_logger)
            
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
            )
        
        self.test_generator(test_generator_logger=evaluation_logger)
        self.score = self.model.evaluate(self.test_generator)
        self.save_score(evaluation_logger)
        
    def save_score(self, save_score_logger: CustomLogger):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("scores.json"), data=scores, logger=save_score_logger)
        
    def log_into_mlflow(self, mlflow_logger: CustomLogger):
        try:
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                    {"loss": self.score[0], "accuracy":self.score[1]}
                )
                # Model Registry does not work with file store
                if tracking_url_type_store != "file":
                    
                    # Register the mode
                    # There are other ways to use the Model Registry, which depends on the use case,
                    # Please refer to the doc for more information:
                    # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                    
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
        
            mlflow_logger.save_logs(msg="MLFlow Logging Successful", log_level="info")
        
        except Exception as e:
            mlflow_logger.save_logs(msg=f"MLFlow Logger failed to execute. Error: {e}", log_level="error")
                    
    

## 7. Update the pipeline

In [ ]:
from src.logger import create_log_path
import logging

# path to save the log files
evaluation_logger_log_file_path = create_log_path("evaluation/evaluation_logger")
evaluation_logger = CustomLogger(logger_name="evaluation_logger", log_filename=evaluation_logger_log_file_path)
evaluation_logger.set_log_level(level=logging.INFO)


mlflow_logger_log_file_path = create_log_path("evaluation/mlflow_logger")
mlflow_logger = CustomLogger(logger_name="mlflow_logger", log_filename=mlflow_logger_log_file_path)
mlflow_logger.set_log_level(level=logging.INFO)


configuration_log_file_path = create_log_path("configuration")
configuration_logger = CustomLogger(logger_name="configuration", log_filename=configuration_log_file_path)
configuration_logger.set_log_level(level=logging.INFO)

############################## ERROR CORRECTION START ##########################
tf.config.run_functions_eagerly(True)
print("Eager Execution:", tf.executing_eagerly())  # Check if enabled

try:
    config = ConfigurationManager(configuration_logger)
    eval_config = config.get_evaluation_config()
    
    evaluation = Evaluation(eval_config)
    evaluation.evaluate(evaluation_logger)
    evaluation.log_into_mlflow(mlflow_logger)
    
    
except Exception as e:
    raise e

INFO:configuration:yaml file: config\config.yaml loaded successfully


INFO:configuration:yaml file: params.yaml loaded successfully
INFO:configuration:created directory at: models


Eager Execution: True


INFO:evaluation_logger:Model loaded from models/training/model.h5 successfully.


Found 139 images belonging to 2 classes.


INFO:evaluation_logger:Generated Validation generator successfully with these 
datagenerator_kwargs: {'rescale': 0.00392156862745098, 'validation_split': 0.3},
dataflow_kwargs: ({'directory': 'data/raw/kidney-ct-scan-image', 'subset': 'validation', 'shuffle': False, 'class_mode': 'categorical'}, {'target_size': [224, 224], 'batch_size': 32, 'interpolation': 'bilinear'})


5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.7247 - loss: 0.6123


INFO:evaluation_logger:json file saved at: scores.json
2025/03/20 02:21:54 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/03/20 02:22:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/03/20 02:25:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.


🏃 View run intelligent-koi-632 at: https://dagshub.com/SaadMalik0011/Kidney-Disease-Classification-With-MLOps.mlflow/#/experiments/0/runs/1e120394caec405b80ac4aa8775e9838
🧪 View experiment at: https://dagshub.com/SaadMalik0011/Kidney-Disease-Classification-With-MLOps.mlflow/#/experiments/0


INFO:mlflow_logger:MLFlow Logging Successful
